## Getting started

**about batch**

> Since there's a finite amount of comments, it is much easier to build a batch based on target keyword features. Here we fetch three different databases sequentially (Duplicates if any, are removed automatically).

## Dataset

Fit the batch of queries to a training dataset. It's easy to load from any data source, as long as it's of type -> `Iterable[List[str]]` and use the text preprocessing methods available in the `david_sentiment.dataset` module.

In [1]:
from david_sentiment import YTCSentimentConfig, YTCSentimentModel
import david_sentiment.dataset as ds

Using TensorFlow backend.


In [7]:
config = YTCSentimentConfig(max_strlen=3000, project_dir="ytc_sentiment")

batch = ds.BatchDB([
    ds.Fetch('unbox', "%make a video%"),
    ds.Fetch('v1', "%make a video%"),
])
x_train, x_labels, y_test = ds.fit_batch_to_dataset(batch, config)

Comments: 100%|██████████| 1483/1483 [00:00<00:00, 14271.51/s]

⚠ * Preprocessing batch with 1483 samples...
✔ * Removed 46 comments  from 1483. Returning size: 1437



Docs: 0 [00:00, ?/s]

⚠ * Transforming texts to sentences with en_core_web_sm model.


Docs: 1437 [00:05, 262.38/s]

✔ * Done! Sucessfully preprocessed 3141 sentences.
ℹ * Size before: 1437, and after: 3141.
⚠ * Building to training data...
✔ * Done! x_train: 1132, x_labels: 1132.


## Training

> That's it! we can now train the model.

In [8]:
ytc_sentiment = YTCSentimentModel(config)
ytc_sentiment.train_model(x_train, x_labels)

✔ Loading vocab file from
/home/ego/david_models/glove/glove.6B/glove.6B.100d.txt
✔ num-dim:(100), vocab-size: 2552
✔ *** embedding vocabulary 🤗 ***
Epoch 1/100
1132/1132 [==============================] - 1s 870us/step - loss: 0.6139 - acc: 0.7120
Epoch 2/100
1132/1132 [==============================] - 1s 682us/step - loss: 0.5320 - acc: 0.7447
Epoch 3/100
1132/1132 [==============================] - 1s 727us/step - loss: 0.4891 - acc: 0.7615
Epoch 4/100
1132/1132 [==============================] - 1s 795us/step - loss: 0.4536 - acc: 0.7959
Epoch 5/100
1132/1132 [==============================] - 1s 625us/step - loss: 0.4271 - acc: 0.8030
Epoch 6/100
1132/1132 [==============================] - 1s 744us/step - loss: 0.4040 - acc: 0.8242
Epoch 7/100
1132/1132 [==============================] - 1s 666us/step - loss: 0.3860 - acc: 0.8339
Epoch 8/100
1132/1132 [==============================] - 1s 587us/step - loss: 0.3641 - acc: 0.8507
Epoch 9/100
1132/1132 [============================

#### Demos

**`test_polarity_attention_weights()` + `test_unseen_samples()`**

While its a rather simple test and not an **official** technique to evalue an embedding model. It does helps at detecting (based on own judgement) the balance between the sum of N `P(negative|positive)` token sequences:

> When scores are: *high* -> *low* : `Balanced-model`

```python
input: "I hate this, but love it :)" < pos(😊) (65.7528)% >
input: "I love this, but hate it :)" < pos(😊) (63.7456)% >
input: "I hate this, but love it :(" < neg(😑) (54.1849)% >
input: "I love this, but hate it :(" < neg(😶) (51.9947)% >
```

- Others (Based on experience):
    - Score values have no order (all-over) : `Overfitted-model`
    - Positive texts are labeled as negative : `Bias-samples`
    
> ***TIP*** kewords like `love, hate` or similar increase bias or/and overfitting. Intead try fetching topics, titles e.g., `%make a video%`, where the word-tokens are not based purely on `sentimentalism`.

In [9]:
from david_sentiment.utils import test_polarity_attention_weights, test_unseen_samples

In [10]:
# Small demo - test the model's attention weights.
test_polarity_attention_weights(ytc_sentiment)

input: "I hate this, but love it :)" < pos(😊) (65.7528)% >
input: "I love this, but hate it :)" < pos(😊) (63.7456)% >
input: "I hate this, but love it :(" < neg(😑) (54.1849)% >
input: "I love this, but hate it :(" < neg(😶) (51.9947)% >


In [11]:
# Small demo - run predictions over k random test_samples.
test_unseen_samples(ytc_sentiment, y_test, print_k=50)

💬 (Old=0.0, New=77.6381)
 😁 - m15 /m17 upgraded version.

💬 (Old=0.0, New=73.272)
 😅 - shall we﻿

💬 (Old=0.0, New=30.4416)
 😳 - I thought it was an explanation.

💬 (Old=0.0, New=99.9355)
 😍 - lew make a video on all the pictures your phones taken from u making phone reviews.

💬 (Old=0.0, New=99.3777)
 😍 - A VIDEO OF YOU PLAYING HOCKEY IN BEATS PB PRO

💬 (Old=0.0, New=94.8878)
 🤗 - Thank you

💬 (Old=0.0, New=83.3552)
 😁 - Can you make a video of iPados beta ?

💬 (Old=0.0, New=33.9808)
 😬 - Windows

💬 (Old=0.0, New=17.025)
 😠 - what do you feel about switching to the oneplus

💬 (Old=0.0, New=99.8896)
 😍 - pewdiepie plz u subcribe me and make a video on me

💬 (Old=0.0, New=91.9985)
 😀 - You should make a video of you playing PUBG on this phone.

💬 (Old=0.0, New=48.4672)
 😶 - If it's supposed to be an april fools

💬 (Old=0.0, New=98.5463)
 😍 - Plz make a video on India, cost & religion system....

💬 (Old=0.0, New=52.7096)
 😑 - Plz make a video of redmi note 7 pro

💬 (Old=0.0, New=31.7184)


In [20]:
# predict sentiment with punctuation.

ytc_sentiment.print_predict("hello, world! i am glad this demo worked! :)")

input: "hello, world! i am glad this demo worked! :)" < pos(😍) (98.3824)% >


In [19]:
# predict sentiment without punctuation.

ytc_sentiment.print_predict("hello world I am glad this demo worked")

input: "hello world I am glad this demo worked" < pos(😀) (91.5674)% >


## Save your session and use later

**you wont have to load the dataset again**

> The following method creates a new project. The `YTCSentimentConfig` class uses these files re-load your model and tokenizer AKA `your project`. So you can feel free to save what you have now and continue later!

In [21]:
ytc_sentiment.save_project()